In [1]:
import pandas as pd
import numpy as np
import pickle
import sklearn

from pathlib import Path

In [2]:
# Load data
cwd = Path.cwd()
moondf = pickle.load(open(cwd / '..' / 'raw_data' / 'moonGen_scrape_2016_final_df', 'rb'))

In [3]:
print(moondf.head())

   0_4  0_5  0_8  0_9  0_10  0_11  0_12  0_13  0_14  0_15  ...  10_9  10_10  \
0    0    0    0    0     0     0     0     0     0     0  ...     0      0   
1    0    0    0    0     0     0     0     0     1     0  ...     0      0   
2    0    0    0    0     0     0     0     0     0     0  ...     0      0   
3    0    0    0    0     0     0     0     0     0     0  ...     0      0   
4    0    0    0    0     0     0     0     0     0     0  ...     0      0   

   10_11  10_12  10_13  10_15  10_17  grade  is_benchmark  repeats  
0      0      0      0      0      0     6C         False        1  
1      0      0      0      0      1     7A         False        0  
2      0      0      0      0      0    6B+         False       14  
3      0      0      0      0      0     6C         False       18  
4      0      0      0      0      0    6B+         False      125  

[5 rows x 144 columns]
